<a href="https://colab.research.google.com/github/chungntu/1DCNN-LSTM-ResNet/blob/main/Problem1_A_spring_problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

# ===============================
# Finite Element Analysis (1D spring/bar-like)
# MATLAB -> Python (NumPy)
# ===============================

# elementNodes: connections at elements
# MATLAB: elementNodes=[1 2; 2 3; 2 4];
element_nodes = np.array([
    [1, 2],
    [2, 3],
    [2, 4]
], dtype=int)

number_elements = element_nodes.shape[0]
number_nodes = 4

# For structure:
displacements = np.zeros((number_nodes, 1), dtype=float)   # u
force = np.zeros((number_nodes, 1), dtype=float)           # f
stiffness = np.zeros((number_nodes, number_nodes), dtype=float)  # K

# applied load at node 2
# MATLAB: force(2)=10.0;
force[1, 0] = 10.0   # Python is 0-based, node 2 -> index 1

# computation of the system stiffness matrix
# local stiffness: [1 -1; -1 1]
k_local = np.array([[ 1.0, -1.0],
                    [-1.0,  1.0]])

for e in range(number_elements):
    # MATLAB nodes are 1-based -> convert to 0-based indices
    dof = element_nodes[e] - 1  # e.g., [1,2] -> [0,1]
    i, j = dof[0], dof[1]
    stiffness[np.ix_([i, j], [i, j])] += k_local

# boundary conditions and solution
# MATLAB: prescribedDof=[1;3;4];
prescribed_dof = np.array([1, 3, 4], dtype=int) - 1  # -> [0,2,3]

all_dof = np.arange(number_nodes)
active_dof = np.setdiff1d(all_dof, prescribed_dof)   # free dof

# solve: K_ff * u_f = f_f
K_ff = stiffness[np.ix_(active_dof, active_dof)]
f_f  = force[active_dof, :]

u_f = np.linalg.solve(K_ff, f_f)

# positioning all displacements
u = np.zeros((number_nodes, 1))
u[active_dof, :] = u_f

# reactions: r = K*u - f  (only meaningful on prescribed dof)
reactions = stiffness @ u - force

# ===== output (like outputDisplacementsReactions) =====
print("K (global stiffness):\n", stiffness)
print("\nForce vector f:\n", force)
print("\nDisplacements u:\n", u)

print("\nReactions (K*u - f):\n", reactions)
print("\nReactions at prescribed dof:")
for nd in (prescribed_dof + 1):  # back to 1-based for display
    print(f"  Node {nd}: R = {reactions[nd-1,0]:.6g}")


K (global stiffness):
 [[ 1. -1.  0.  0.]
 [-1.  3. -1. -1.]
 [ 0. -1.  1.  0.]
 [ 0. -1.  0.  1.]]

Force vector f:
 [[ 0.]
 [10.]
 [ 0.]
 [ 0.]]

Displacements u:
 [[0.        ]
 [3.33333333]
 [0.        ]
 [0.        ]]

Reactions (K*u - f):
 [[-3.33333333]
 [ 0.        ]
 [-3.33333333]
 [-3.33333333]]

Reactions at prescribed dof:
  Node 1: R = -3.33333
  Node 3: R = -3.33333
  Node 4: R = -3.33333
